# Q1: Commuter pattern

### Question: Do average hourly rides differ between working days and non-working days?

1. Which stakeholder cares?
- Ops Lead = staffing & rebalancing windows differ by weekday vs weekend
- PM = validates commuter vs leisure hypothesis

2. How to sample this data?
- Take all rows (hours) labeled workingday = 1 (weekdays) and workingday = 0 (weekends/holidays).
- Compare their mean ride counts (cnt).
- Each row is an independent hourly observation... so it's  perfect for this test.

3. My hypothesis
- Null hyp H0: mean rides are the same on working days and non working days
- Alt hyp H1: mean rides are different between working days and non working days

4. Which test?
- This would be a two sample independent test becasue we are comparing means across two independent groups aka weekends vs weekdays 

5. My Alpha test
- .5 sig level b/c it balances the risk of false positives vs missing a real effect.

6. Explaining the result 

(test name + results (p-value, CI), decision (reject or not), plain-eng meaning (why it matters))

- Run t-test = report p-value & 95% confidence interval of the mean difference.
- If p < 0.05 =vreject H₀: conclude average hourly rides are significantly different.
- Interpretation would be that the practical significance: weekday vs weekend usage follows different demand patterns (commute vs leisure).

In [3]:
import pandas as pd
import numpy as np
from scipy import stats


In [4]:
df = pd.read_csv('/Users/ayemaq/Desktop/Mod4_prj/dataset/hour.csv')

In [5]:
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [6]:
# split rides into workingday vs non-workingday
workday_rides = df[df['workingday'] == 1]['cnt']
nonworkday_rides = df[df['workingday'] == 0]['cnt']

In [7]:
# run the welch's t-test
t_stat, p_value = stats.ttest_ind(workday_rides, nonworkday_rides, equal_var=False)
print(f"T-statistic: {t_stat}, P-value: {p_value}")


T-statistic: 4.095067721524371, P-value: 4.249478377549554e-05


In [8]:
# difference in means
mean_diff = workday_rides.mean() - nonworkday_rides.mean() #11.8 is about 12 extra rides/hour on average

# standard error of difference
se_diff = np.sqrt(workday_rides.var()/len(workday_rides) + nonworkday_rides.var()/len(nonworkday_rides))

# 95% CI
ci_low = mean_diff - 1.96*se_diff
ci_high = mean_diff + 1.96*se_diff

print("The Mean Difference:", mean_diff)
print("95% CI:", (ci_low, ci_high))


The Mean Difference: 11.802422015538411
95% CI: (6.15349293217658, 17.451351098900243)


## Result Of Part A 
- A two-sample Welch’s t-test found a significant difference in average hourly rides between working days and non-working days (t=4.10, p<0.001). 
- Conclusion: weekdays really do have more rides per hour than weekends,  about 12 extra rides/hour on average. we’re extremely confident it’s real, not random


Stakeholder interpretation:
- PM = confirms commuter-driven demand.
- Ops Lead = weekdays need extra staffing and bike rebalancing.
- Marketing = focus weekend promos on casual riders, since weekday demand is already strong.

It is important to understand the data so far: 

1. Weekdays (working days):
    - Strong commuter peaks (8am, 5–6pm).
    - Registered riders dominate (members, locals).

2. Weekends/non-working days:
    - Flatter, smoother demand curve.
    -  driven by casual riders (tourists, leisure, one-time users).


Why I said “focus on casual riders for weekends” for stakeholder interpretation above (marketing) 
- From a Marketing lens:
    - Registered riders are already locked in (commuters who use the system daily).

- Casual riders are less predictable, more influenceable by promos.
    - So, promos on weekends are more likely to convert occasional riders → bigger ROI.

However! 

- It doesn’t mean ignore registered riders.xWe should also focus on weekends, because people who work during the week may want to relax and enjoy their day off in a meaningful way. Weekends represent an opportunity for both casual riders (tourists, leisure users) and registered riders (locals using bikes for recreation or errands).
- Two nuances:
    1. PM / Ops side: must still plan capacity for both groups (registered commuters + weekend leisure).
    2. Marketing side: messaging/targeting may emphasize casuals on weekends because they’re the growth opportunity.


so the split is really about which lever matters most for each stakeholder:
- Ops: serve both: enough bikes & docks for all patterns.
- PM: track both: confirm system is reliable across use cases.

# Part B: Multi-group comparison
Q: Do mean hourly rides differ across categories of multi-level categorical variables such as season or weather condition (choose one)? If you find a difference, describe the appropriate post-hoc (after the test what other tests would you do) approach and what it
would tell stakeholders)

weather condition (weathersit) is categorical with 4 levels:

- 1 = Clear / Few clouds
- 2 = Mist / Cloudy
- 3 = Light Snow / Light Rain
- 4 = Heavy Rain / Storm

- H0 (null): all weather conditions -> same mean rides/hour
- H1 (Alt): at least one group filter 
- Run ANOVA: get F-stat (ratio of between-group variance / within-group variance).
    - Large F → groups differ beyond random chance.
    - Small F → differences are just noise.
- Check p-value: if < 0.05 → reject H0 → not all means are equal.
    - BUT… ANOVA doesn’t tell you which differ.
- So run Tukey HSD post-hoc to see pairwise comparisons (clear vs mist, clear vs storm, etc.).

### Framing
Stakeholders: 
- PM & Ops -  plan capacity by weather; choose guardrails for bad-weather days.
- Marketing: avoid promos during storms; lean into clear days.

Sampling:
- Use all hourly rows. Split by weathersit (1=Clear, 2=Mist/Cloudy, 3=Light snow/rain, 4=Heavy rain/storm). Outcome = cnt.

Hypotheses:
- H0: u1 = u2 = u3- u4 -> Thism eans ride/hour are = across weather categories 
- H1: at least one u differs 

Alpha:
- .05 sig 

In [11]:
clear = df[df['weathersit'] == 1]['cnt']
mist  = df[df['weathersit'] == 2]['cnt']
light = df[df['weathersit'] == 3]['cnt']
heavy = df[df['weathersit'] == 4]['cnt']


In [ ]:
print("Clear:", clear.shape[0], "mean =", clear.mean())
print("Mist :", mist.shape[0], "mean =", mist.mean())
print("Light:", light.shape[0], "mean =", light.mean())
print("Heavy:", heavy.shape[0], "mean =", heavy.mean())

# n (shape[0])... how many hourly rows are in that group


Clear: 11413 mean = 204.8692718829405
Mist : 4544 mean = 175.16549295774647
Light: 1419 mean = 111.57928118393235
Heavy: 3 mean = 74.33333333333333


Clear (n=11,413): mean ≈ 205 rides/hour

Mist (n=4,544): mean ≈ 175 rides/hour

Light Snow/Rain (n=1,419): mean ≈ 112 rides/hour

Heavy Rain/Storm (n=3): mean ≈ 74 rides/hour

In [13]:
F, p = stats.f_oneway(clear, mist, light, heavy)
print("F-statistic:", F, "p-value:", p)


F-statistic: 127.17386949967266 p-value: 1.7347820521802623e-81


ANOVA only tells us “at least one group is different.” To see which groups differ (Clear vs Mist, Clear vs Light Rain, etc.), we need to run the Tukey HSD post-hoc test:

In [14]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(endog=df['cnt'], groups=df['weathersit'], alpha=0.05)
print(tukey.summary())


  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower    upper   reject
--------------------------------------------------------
     1      2  -29.7038    0.0  -37.7909 -21.6166   True
     1      3    -93.29    0.0 -106.2676 -80.3123   True
     1      4 -130.5359 0.5886 -396.7534 135.6815  False
     2      3  -63.5862    0.0  -77.6067 -49.5658   True
     2      4 -100.8322 0.7649 -367.1025 165.4381  False
     3      4  -37.2459 0.9841 -303.7096 229.2177  False
--------------------------------------------------------


ANOVA confirmed that weather conditions significantly affect mean hourly rides (F=127.17, p<0.001). Tukey HSD revealed that clear conditions have significantly higher ridership than mist or light snow/rain, and misty conditions also outperform light snow/rain. Comparisons involving heavy rain/storm were not significant due to very limited data (n=3).

In [ ]:
df.groupby('weathersit')['cnt'].agg(['count','mean'])

,count,mean
weathersit,,
1,11413,204.869272
2,4544,175.165493
3,1419,111.579281
4,3,74.333333


In [16]:
weather_summary = (
    df.groupby('weathersit')['cnt']
      .agg(['count','mean','std'])
      .assign(
          se=lambda x: x['std'] / np.sqrt(x['count']),     # standard error
          ci_low=lambda x: x['mean'] - 1.96 * x['se'],     # lower bound
          ci_high=lambda x: x['mean'] + 1.96 * x['se']     # upper bound
      )
)

print(weather_summary)


            count        mean         std         se      ci_low     ci_high
weathersit                                                                  
1           11413  204.869272  189.487773   1.773705  201.392811  208.345733
2            4544  175.165493  165.431589   2.454140  170.355379  179.975607
3            1419  111.579281  133.781045   3.551431  104.618476  118.540086
4               3   74.333333   77.925178  44.990122  -13.847307  162.513973


## Results for part B

### Setup
I tested whether mean hourly rides differ across weather conditions (clear, mist, light snow/rain, heavy rain/storm) using a one-way ANOVA with α = 0.05. The null hypothesis assumed equal means across all conditions, while the alternative allowed at least one group to differ.

### Results
The ANOVA was significant (F = 127.18, p < 0.001), indicating that ridership varies by weather. Tukey post-hoc tests showed clear days had significantly higher ridership than mist (≈30 more rides/hour) and light snow/rain (≈93 more rides/hour). Mist also outperformed light snow/rain (≈64 more rides/hour). Comparisons with heavy rain/storm were not significant, but this likely reflects the very small sample size (n = 3).

### Group Averages
Average ridership was highest on clear days (205 rides/hour, 95% CI [201, 208]), moderate on misty days (175 rides/hour, CI [170, 180]), and much lower during light snow/rain (112 rides/hour, CI [105, 118]). Stormy conditions averaged 74 rides/hour, but estimates are unreliable due to the tiny sample.

### Interpretation
Weather has a substantial effect on bike demand. Clear days consistently generate the strongest usage, while precipitation reduces ridership. This means operations teams should expect lower demand and adjust staffing/rebalancing accordingly, marketing should focus promotions on clear days, and PMs should treat weather as a confounding factor in testing new features.

